In [1]:
!nvidia-smi

Fri Jul 10 19:14:57 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  On   | 00000000:65:00.0  On |                  N/A |
| 23%   41C    P0    60W / 250W |    591MiB / 11170MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
from utils import load_data
from embedding import get_token_data, get_s2v_model, get_p2v_model

In [233]:
train, val, song_meta = load_data()

In [19]:
train.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000
3,"[연말, 눈오는날, 캐럴, 분위기, 따듯한, 크리스마스캐럴, 겨울노래, 크리스마스,...",147456,크리스마스 분위기에 흠뻑 취하고 싶을때,"[394031, 195524, 540149, 287984, 440773, 10033...",33,2019-12-05 15:15:18.000
4,[댄스],27616,추억의 노래 ㅋ,"[159327, 553610, 5130, 645103, 294435, 100657,...",9,2011-10-25 13:54:56.000


In [7]:
total = get_token_data(train, val)

In [9]:
s2v_model = get_s2v_model(total, 2, 100, 3, 5)

In [10]:
p2v_model = get_p2v_model(train, val, s2v_model)

In [14]:
from gensim.models import Word2Vec 
s2v_model = Word2Vec.load("./manual_emb/s2v_mdl_2020-07-08T21:17:03.model")

In [17]:
from gensim.models import Word2Vec, KeyedVectors
p2v_model = KeyedVectors.load("./manual_emb/p2v_mdl_2020-07-08T21:17:26.model", mmap='r')

In [18]:
p2v_model.most_similar("6362")

[('17095', 0.9961541891098022),
 ('98472', 0.9928523302078247),
 ('86701', 0.9927220344543457),
 ('63269', 0.9918633699417114),
 ('84156', 0.9918058514595032),
 ('79909', 0.9914500713348389),
 ('102725', 0.9913939833641052),
 ('5960', 0.9911588430404663),
 ('96197', 0.9911543130874634),
 ('68932', 0.9907352924346924)]

In [236]:
tags_input = ["추억", "회상"]
songs_input = [394031, 394031] 

In [57]:
tmp_list = get_input_df(tags_input, songs_input)

In [249]:
# get_input_df
import numpy as np
from collections import Counter

### input setting
def get_input_df(tags, songs):
    return tags + [str(x) for x in songs]

### word2vec model loading
def get_s2v_mdl(file_path):
    from gensim.models import Word2Vec 
    return Word2Vec.load(file_path)

def get_p2v_mdl(file_path):
    from gensim.models import Word2Vec, KeyedVectors
    return KeyedVectors.load(file_path, mmap='r')

### token to vector
def get_song_tag_vec(token, s2v_mdl):
    try:
        return s2v_mdl.wv.word_vec(token)
    except:
        return np.array([0.0]*100)
    
### sum of token vector(=playlist vector)
def get_plylst_vec(token_list, s2v_mdl):
    plylst_vec = None
    for idx_, x in enumerate(token_list):
        if idx_ ==0:
            plylst_vec = get_song_tag_vec(x, s2v_mdl)
        else : 
            plylst_vec =  plylst_vec + get_song_tag_vec(x, s2v_mdl)
    return plylst_vec

### plylist to similar plylist id
def get_most_similar_plylst(plylst_vec, p2v_mdl, topn = 20):
    return p2v_mdl.similar_by_vector(plylst_vec, topn=topn, restrict_vocab=None)

### plylist id to songs
def get_songs_from_plylst(df, id_, korean=False, song_meta=None):
    if korean == False:
        return list(df[df["id"]==int(id_)]["songs"])[0]
    else :
        songs = list(df[df["id"]==int(id_)]["songs"])
        return [song_meta[song_meta['id']==x]["song_name"].item() for x in songs[0]]
    
def get_tags_from_plylst(df, id_):
     return list(df[df["id"]==int(id_)]["tags"])[0]

### MS top20 playlist songlist
def get_MS_song_freq(df, song_meta, plylst_vec, p2v_mdl, topn=20, korean=False):
    MS_plylst = get_most_similar_plylst(plylst_vec, p2v_mdl, topn)
    
    total_songs = [] 
    for plylst in MS_plylst:
        songs = get_songs_from_plylst(df, plylst[0], korean=korean, song_meta=song_meta)
        total_songs += songs
        
    return total_songs

def get_MS_tag_freq(df, plylst_vec, p2v_mdl, topn=20):
    MS_plylst = get_most_similar_plylst(plylst_vec, p2v_mdl, topn)
    
    total_songs = [] 
    for plylst in MS_plylst:
        songs = get_tags_from_plylst(df, plylst[0])
        total_songs += songs
        
    return total_songs

def get_count_list(token_list):
    count_list = sorted(dict(Counter(token_list)).items(), key=(lambda x :x[1]), reverse=True)
    return [x for x in count_list if x[1]>1]

In [ ]:
s2v_file_path = "./manual_emb/s2v_mdl_2020-07-08T21:17:03.model"
p2v_file_path = "./manual_emb/p2v_mdl_2020-07-08T21:17:26.model"

In [283]:
total_df = pd.concat([train, val], axis=0)
total_df = total_df.reset_index()

def base_model(tags_input, songs_input):

    song_count_list = []
    tag_count_list = []
    
    if len(tags_input + songs_input) !=0:

        tmp_list = get_input_df(tags_input, songs_input)

        s2v_mdl =get_s2v_mdl(s2v_file_path)
        plylst_vec = get_plylst_vec(tmp_list, s2v_mdl)

        p2v_mdl = get_p2v_mdl(p2v_file_path)

        song_list = get_MS_song_freq(total_df, song_meta, plylst_vec, p2v_mdl, korean= False, topn=200)
        song_count_list = get_count_list(song_list)

        tag_list = get_MS_tag_freq(total_df, plylst_vec, p2v_mdl, topn=200)
        tag_count_list = get_count_list(tag_list)  
        
        add_songs = max(0, 100-len(songs_input))
        add_tags = max(0, 10-len(tags_input))
        
        if len(song_count_list) <= add_songs:
            pass
        else:
            song_count_list = song_count_list[:add_songs]
            
        song_count_list = [x[0] for x in song_count_list]
        song_count_list += songs_input
        
        if len(tag_count_list) <= add_tags:
            pass
        else:
            tag_count_list = tag_count_list[:add_tags]
            
        tag_count_list = [x[0] for x in tag_count_list]
        tag_count_list += tags_input


    else:
        print("empty tags and songs")
    
    return [song_count_list, tag_count_list]

In [ ]:
#tmp["songs_base_result"],  tmp["tags_base_result"]= tmp.apply(lambda x : base_model(x["tags"], x["songs"]), axis=1)
val_result = val.apply(lambda x : base_model(x["tags"], x["songs"]), axis=1)

empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs


In [ ]:
val["tags_result"] = [x[1] for x in val_result] 
val["songs_result"] = [x[0] for x in val_result] 

In [ ]:
val[["id", "tags_result", "songs_result"]].to_csv("base_list.csv", encoding="cp949")

In [104]:
from collections import Counter
result_dict = dict(Counter(songs_total))
sorted(result_dict.items(), key=(lambda x :x[1]), reverse=True)



[(116847, 14),
 (284814, 9),
 (424869, 8),
 (177284, 8),
 (201145, 8),
 (582905, 8),
 (669617, 8),
 (143652, 7),
 (356571, 7),
 (397924, 7),
 (485957, 7),
 (149779, 7),
 (4412, 7),
 (555134, 6),
 (591047, 6),
 (367459, 6),
 (347303, 6),
 (433447, 6),
 (21865, 6),
 (438086, 6),
 (237282, 6),
 (380859, 6),
 (87752, 5),
 (633203, 5),
 (70662, 5),
 (158554, 5),
 (298646, 5),
 (262145, 5),
 (592386, 5),
 (16847, 5),
 (68266, 4),
 (199390, 4),
 (411756, 4),
 (486220, 4),
 (599290, 4),
 (569548, 4),
 (477964, 4),
 (133330, 4),
 (4083, 4),
 (426013, 4),
 (268811, 4),
 (256465, 4),
 (629177, 4),
 (466048, 4),
 (186358, 4),
 (96825, 4),
 (651339, 4),
 (665144, 4),
 (518239, 3),
 (548735, 3),
 (11914, 3),
 (170578, 3),
 (477689, 3),
 (587193, 3),
 (554558, 3),
 (122384, 3),
 (32042, 3),
 (415774, 3),
 (570100, 3),
 (173709, 3),
 (705462, 3),
 (68488, 3),
 (402661, 3),
 (635160, 3),
 (583525, 3),
 (69933, 3),
 (488423, 3),
 (531732, 3),
 (38311, 3),
 (15958, 3),
 (661213, 3),
 (221986, 3),
 (45421

In [137]:
unused_song = []
new_dic = {}
for i in result_dict.items():
    if str(i[0]) not in songs and i[1]>=2:
        print(str(i[0]))
        new_dic[str(i[0])] = i[1]
        
print(sorted(new_dic.items(), key=(lambda x :x[1]), reverse=True))

11914
170578
411756
486220
367459
70662
16083
38581
477689
347303
590379
284814
599290
23570
177284
569548
480590
433447
230729
356571
158554
201145
21865
397924
153734
587193
285445
675126
554558
298646
477964
122384
547977
582905
262145
143877
188590
133330
319297
592386
122998
32042
142498
438086
579395
4083
415774
395858
669617
237282
485957
1099
8187
699365
570100
638346
173709
622190
149779
705462
380859
68488
4412
546064
402661
426013
635160
583525
274820
69933
485142
444479
488423
184540
268811
531732
256465
292655
317710
192195
94820
21408
320382
429256
127614
695494
629177
466048
16847
128242
38311
65958
15958
661213
186358
221986
543556
454218
414755
134728
134583
46284
589242
96825
28024
132689
653071
651339
38836
349886
675797
677063
85800
688951
665144
602629
539892
524168
374617
63874
503911
355713
235609
678294
565
220781
77781
679405
184015
169935
635759
135756
527887
65062
106438
466004
566376
50055
325001
93307
322776
436255
344327
439190
413422
18768
25892
157566
54

In [130]:
new_dic

[]

In [121]:
len(unused_song)

482

In [123]:
len(result_dict.items())

498

In [122]:
len(songs)

16

In [22]:

def get_p2v_array(input_data):
    total_array = None

    for idx_, token in enumerate(input_data):

        if token in s2v_model.wv.vocab:

            if idx_==0:
                total_array = s2v_model.wv.word_vec(token)
            else:
                total_array = total_array + s2v_model.wv.word_vec(token)

        else:
            pass
        
    return total_array